# Load Data from apple_health_export/ 
Originally as XML file, want to convert to csv

In [1]:
# Load needed packages
import pandas as pd
import xmltodict

In [2]:
# use your own directory, this is where I exported my data
data_path = 'apple_health_export/export.xml' 

# read XML file, convert to a dictionary
with open(data_path, 'r') as xml_file:
    all_data = xmltodict.parse(xml_file.read())

In [3]:
# look at all data available (remember it is in a dictionary)
all_data['HealthData'].keys()

odict_keys(['@locale', 'ExportDate', 'Me', 'Record', 'Workout', 'ActivitySummary'])

For now, I will just show the steps for cleaning my individual activity summary (acts), which I will put into a list, and then pandas dataframe. I will also do so for other records (all), workout, etc. in the future

In [5]:
# create a records list for overall health data, put into pandas dataframe
records_list = all_data['HealthData']['Record']
all_records = pd.DataFrame(records_list)

In [6]:
# activity summary (acts)
acts_list = all_data['HealthData']['ActivitySummary']
acts_df = pd.DataFrame(acts_list)

In [7]:
# glimpse at data
acts_df.head()

,@dateComponents,@activeEnergyBurned,@activeEnergyBurnedGoal,@activeEnergyBurnedUnit,@appleMoveTime,@appleMoveTimeGoal,@appleExerciseTime,@appleExerciseTimeGoal,@appleStandHours,@appleStandHoursGoal
0,1969-12-30,0,0,Cal,0,0,0,30,0,12
1,1969-12-31,0,0,Cal,0,0,0,30,0,12
2,2015-08-29,0,0,Cal,0,0,0,30,0,12
3,2015-08-30,0.467,0,Cal,0,0,0,30,0,12
4,2015-10-01,0,0,Cal,0,0,0,30,0,12


# Now that data is imported, I will clean it for easier use 

In [8]:
## Cleaning 

# remove special characters from column names 
acts_df.columns = acts_df.columns.str.replace('@', '')

# convert date column to date format
acts_df['dateComponents'] = pd.to_datetime(acts_df['dateComponents'])

# make sure dates are only within range from 1/2016-recent (4/2022),
# this is when I got the apple watch, data is not just from phone
acts_df = acts_df[(acts_df['dateComponents'] > '2016-1-1') & (acts_df['dateComponents'] <= '2022-4-1')]


In [9]:
acts_df.head()

,dateComponents,activeEnergyBurned,activeEnergyBurnedGoal,activeEnergyBurnedUnit,appleMoveTime,appleMoveTimeGoal,appleExerciseTime,appleExerciseTimeGoal,appleStandHours,appleStandHoursGoal
97,2016-01-02,564.696,320,Cal,0,0,49,30,12,12
98,2016-01-03,805.292,320,Cal,0,0,73,30,12,12
99,2016-01-04,791.03,350,Cal,0,0,73,30,14,12
100,2016-01-05,206.855,350,Cal,0,0,5,30,12,12
101,2016-01-06,617.195,350,Cal,0,0,54,30,13,12


Now that dates and columns are done, lets look at data types

In [10]:
acts_df.dtypes

dateComponents            datetime64[ns]
activeEnergyBurned                object
activeEnergyBurnedGoal            object
activeEnergyBurnedUnit            object
appleMoveTime                     object
appleMoveTimeGoal                 object
appleExerciseTime                 object
appleExerciseTimeGoal             object
appleStandHours                   object
appleStandHoursGoal               object
dtype: object

In [11]:
# convert the rest of the columns to integers (except for Unit)
cols = acts_df.columns
acts_df[cols[1:3]] = acts_df[cols[1:3]].apply(pd.to_numeric, errors='coerce')
acts_df[cols[4:]] = acts_df[cols[4:]].apply(pd.to_numeric, errors='coerce')

# round active energy so it is consistent
acts_df = acts_df.round()

# For more analyses and visualizations, I will also clean the workouts and put into a dataframe

In [87]:
# create workouts df
workouts_list = all_data['HealthData']['Workout']
workout_df = pd.DataFrame(workouts_list)

In [88]:
# look at variables (workout)
workout_df.keys()

Index(['@workoutActivityType', '@duration', '@durationUnit', '@totalDistance',
       '@totalDistanceUnit', '@totalEnergyBurned', '@totalEnergyBurnedUnit',
       '@sourceName', '@sourceVersion', '@creationDate', '@startDate',
       '@endDate', 'MetadataEntry', 'WorkoutEvent', '@device', 'WorkoutRoute'],
      dtype='object')

In [89]:
# remove special characters from column names 
all_records.columns = all_records.columns.str.replace('@', '')
workout_df.columns = workout_df.columns.str.replace('@', '')

In [90]:
workout_df['workoutActivityType'].unique()

# convert/string replace for easier naming
workout_df['workoutActivityType'] = workout_df['workoutActivityType'].str.replace('HKWorkoutActivityType', '')

In [91]:
# What columns do we have to work with? Can decide how to parse data 
all_records['type'].unique()
workout_df['workoutActivityType'].unique()

array(['Walking', 'Running', 'Elliptical', 'Cycling', 'Rowing',
       'MixedCardio', 'FunctionalStrengthTraining',
       'PreparationAndRecovery', 'HighIntensityIntervalTraining', 'Yoga',
       'Other', 'CrossTraining'], dtype=object)

In [92]:
# convert date column to date format for three columns containing 'date'
# make sure dates are only within range from 1/2016-recent (4/2022),
dates = ['startDate', 'endDate', 'creationDate'] # list of cols with date in them 
for cols in dates:
    workout_df[cols] = pd.to_datetime(workout_df[cols])
    workout_df = workout_df[(workout_df[cols] > '2016-1-1') & (workout_df[cols] <= '2022-4-1')]


In [93]:
#drop columns we do not need
workout_df = workout_df.drop(columns=['MetadataEntry', 'WorkoutEvent', 'device', 'WorkoutRoute'])

In [94]:
# look at data types
workout_df.dtypes

for cols in ['duration', 'totalDistance', 'totalEnergyBurned']:
    workout_df[cols] = pd.to_numeric(workout_df[cols], errors='coerce')

In [96]:
# quickly view data
workout_df.tail()

,workoutActivityType,duration,durationUnit,totalDistance,totalDistanceUnit,totalEnergyBurned,totalEnergyBurnedUnit,sourceName,sourceVersion,creationDate,startDate,endDate
1009,Walking,33.281538,min,2.131079,mi,194.725963,Cal,Rebecca’s Apple Watch,8.4.1,2022-02-10 17:43:56-04:00,2022-02-10 17:10:38-04:00,2022-02-10 17:43:55-04:00
1010,Walking,29.858132,min,2.011422,mi,193.564586,Cal,Rebecca’s Apple Watch,8.4.2,2022-02-12 17:30:30-04:00,2022-02-12 17:00:38-04:00,2022-02-12 17:30:30-04:00
1016,Walking,30.295698,min,2.012164,mi,191.064238,Cal,Rebecca’s Apple Watch,8.4.2,2022-02-23 17:58:53-04:00,2022-02-23 17:28:34-04:00,2022-02-23 17:58:52-04:00
1023,Walking,23.023239,min,1.326645,mi,164.271505,Cal,Rebecca’s Apple Watch,8.4.2,2022-03-08 16:10:07-04:00,2022-03-08 15:47:05-04:00,2022-03-08 16:10:07-04:00
1025,Walking,40.164821,min,2.694165,mi,296.217819,Cal,Rebecca’s Apple Watch,8.4.2,2022-03-25 17:41:12-04:00,2022-03-25 17:01:02-04:00,2022-03-25 17:41:12-04:00


# Now that data is consistent, will write to csv and use from here on out

In [ ]:
# activity summary
acts_df.to_csv("apple_health_export/activitysummary.csv")

In [97]:
# workout
workout_df.to_csv("apple_health_export/workouts.csv")

In [99]:
# all records
# NOTE not all data is cleaned, will need to re-format if using this in the future
all_records.to_csv("apple_health_export/all_records.csv")